# Quick exporatory Notebook to look at running/processing Fluvial-Particle simulations in a notebook.

In [ ]:
%%capture cap --no-stderr
import time

from fluvial_particle import Settings
from fluvial_particle import simulate

argdict = {
    "settings_file": "../tests/data/user_options_test.py",
    "output_directory": "../tests/data/output",
    "seed": None,
    "no_postprocess": True,
}
settings_file = argdict["settings_file"]
options = Settings.read(settings_file)
simulate(options, argdict, timer=time.time)


In [ ]:
from threading import Thread
import pyvista as pv
import pyvistaqt as pvqt
import meshio
import vtk
import numpy as np
import h5py
pv.set_jupyter_backend('panel')

# pyvistaqt Crashes
* Testing pyvistaqt crashes kernal
* un-comment cell below to see

In [ ]:
# import pyvista as pv
# from pyvistaqt import BackgroundPlotter

# sphere = pv.Sphere()

# plotter = BackgroundPlotter()
# plotter.add_mesh(sphere)
# plotter.sho()

# # can now operate on the sphere and have it updated in the background
# # sphere.points *= 0.5

# XDMF
* pyvista uses meshio to read xdmf files.  However, meshio can not read our .xmf files (see cell below).
* comment in meshio issues suggest xdmf is not well supported, don't know how valid the statement is: https://github.com/nschloe/meshio/issues/1273#issue-1117218893
* Interestingly paraview has an XDMF reader that works!

In [ ]:
meshio.read('../tests/data/output/particles.xmf')

# Particle helper Functions
* pretty straight-forward to write our own particle helper functions

In [ ]:
def get_h5file(filename: str) -> h5py._hl.files.File:
    return h5py.File(filename)

def get_num_timesteps(f: h5py._hl.files.File) -> int:
    return f["coordinates"].get("x").shape[0]

def get_points(f: h5py._hl.files.File, time: int, twod: bool=False) -> np.ndarray:
    # f = h5py.File(filename)
    pt_dim = f["coordinates"].get("x").shape[1]
    x = f["coordinates"].get("x")
    y = f["coordinates"].get("y")
    z = f["coordinates"].get("z")
    if not twod:
        return np.array([[[x[i,j], y[i,j], z[i,j]] for i in [time] for j in np.arange(pt_dim)]][0])
    else:
        # returns z as 0.5 so it sits above 2d mesh
        return np.array([[[x[i,j], y[i,j], 0.5] for i in [time] for j in np.arange(pt_dim)]][0])

# Example files
* particle.h5 file generated by running code in fist code block above

In [ ]:
my_mesh = pv.read('../tests/data/Result_FM_MEander_1_long_2D1.vtk')
file = get_h5file('../tests/data/output/particles.h5')


# Inline animation
* https://panel.holoviz.org/reference/panes/VTK.html#panes-gallery-vtk

In [ ]:
import panel as pn
pn.extension('vtk')
pv.start_xvfb()
pl = pv.Plotter()
pl.add_mesh(my_mesh, scalars="IBC")
points = get_points(file, 0, twod=True)
point_cloud = pv.PolyData(points)
pl.add_mesh(point_cloud, point_size=3)
sim_pan = pn.panel(pl.ren_win, width=1000, orientation_widget=True)
nts = get_num_timesteps(file)
print(nts)
player = pn.widgets.Player(name='Player', start=0, end=nts-1, loop_policy='loop', interval=nts-1)
@pn.depends(value=player.param.value)
def newpts(value):
#     print(value)
    points = get_points(file, value, twod=True)
    point_cloud.overwrite(pv.PolyData(points))
#     time.sleep(0.5)
    sim_pan.synchronize()
    
pn.Column(sim_pan, player, newpts).servable()
# sim_pan

# Create animation .mp4
* https://docs.pyvista.org/examples/02-plot/movie.html#sphx-glr-examples-02-plot-movie-py

In [ ]:
plotter = pv.Plotter()
plotter.open_movie("../tests/data/ouput/test.mp4")
# plotter.view_xy()
plotter.add_mesh(my_mesh, scalars="IBC")
points = get_points(file, 0, twod=True)
point_cloud = pv.PolyData(points)
plotter.add_mesh(point_cloud, point_size=3, cmap='r' )

# plotter.camera_postion = [(732.7206856912, 509.9372768109662, 509.7542638486249),
#  (223.21642184257507, 0.4330129623413086, 0.25),
#  (0.0, 0.0, 1.0)]

# plotter.show(auto_close=False)
plotter.write_frame()

def newpts(index):
    points = get_points(file, index, twod=True)
    point_cloud.overwrite(pv.PolyData(points))
#     plotter.render()
    plotter.write_frame()
    

for i in range(3600):
    newpts(i)
    if (i % 100) == 0:
        print(i)
    
plotter.close()